##  ewf-ext-03-02-01 - SeaEyes

...

### <a name="quicklink">Quick link

* [Objective](#objective)
* [Data](#data)
* [Service Definition](#service)
* [Parameter Definition](#parameter)
* [Runtime Parameter Definition](#runtime)
* [Workflow](#workflow)
* [License](#license)

### <a name="objective">Objective 

...

### <a name="data">Data 

...

### <a name="service">Service Definition

In [ ]:
service = dict([('title', 'ewf-ext-03-02-01 - SeaEyes'),
                ('abstract', 'ewf-ext-03-02-01 - SeaEyes'),
                ('id', 'ewf-ext-03-02-01')])

### <a name="parameter">Parameter Definition 

**Background Window Size**

Background Window Size: The window size in pixels for computing local mean backscatter level.

In [ ]:
backgroundWindowSize = dict([('id', 'backgroundWindowSize'),
                             ('value', '181'),
                             ('title', 'Background Window Size'),
                             ('abstract', 'Background Window Size: The window size in pixels for computing local mean backscatter level.'),
                             ('minOccurs', '1')])

**Threshold Shift**

Threshold Shift (dB): The detecting threshold is lower than the local mean backscatter level by this amount.

In [ ]:
thresholdShift = dict([('id', 'thresholdShift'),
                       ('value', '2.0'),
                       ('title', 'Threshold Shift'),
                       ('abstract', 'Threshold Shift (dB): The detecting threshold is lower than the local mean backscatter level by this amount.'),
                       ('minOccurs', '1')])

**Minimum Cluster Size**

The minimum cluster size in square kilometer. Cluster with size smaller than this size is eliminated.

In [ ]:
minClusterSizeInKm2 = dict([('id', 'minClusterSizeInKm2'),
                            ('value', '0.1'),
                            ('title', 'Minimum Cluster Size'),
                            ('abstract', 'The minimum cluster size in square kilometer. Cluster with size smaller than this size is eliminated.'),
                            ('minOccurs', '1')])

**Region Of Interest**

WKT Polygon for the Region of Interest

In [ ]:
regionOfInterest = dict([('id', 'regionOfInterest'),
                         ('value', 'POLYGON ((-54.3452413212612 69.74156437889997, -51.65178350038967 69.98563658328254, -52.514956886601865 71.11255404659198, -55.35237767944321 70.857913012771, -54.3452413212612 69.74156437889997))'),
                         ('title', 'WKT Polygon for the Region of Interest'),
                         ('abstract', 'Set the value of WKT Polygon')])

### <a name="runtime">Runtime parameter definition

**Input identifier**

In [ ]:
input_identifier = 'S1B_IW_GRDH_1SDV_20170703T194823_20170703T194848_006328_00B202_5554'

**Input reference**

In [ ]:
input_reference = 'https://catalog.terradue.com/sentinel1/search?uid=S1B_IW_GRDH_1SDV_20170703T194823_20170703T194848_006328_00B202_5554'

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = '/workspace/data/S-1'

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [ ]:
import snappy

import sys
import os
from py_snap_helpers import op_help, get_operator_default_parameters, GraphProcessor

import cioppy

import numpy as np

import gdal

import datetime

import shutil

#### Methods

In [ ]:
def vessel_detection_processing(**kwargs):
   
    options = dict()
    
    operators = ['Read',
                 'Land-Sea-Mask',
                 'Calibration',
                 'AdaptiveThresholding',
                 'Object-Discrimination',
                 'Write']
    
    for operator in operators:
            
        print 'Getting default values for Operator {}'.format(operator)
        parameters = get_operator_default_parameters(operator)
        
        options[operator] = parameters

    for key, value in kwargs.items():
        
        print 'Updating Operator {}'.format(key)
        options[key.replace('_', '-')].update(value)
    
    mygraph = GraphProcessor()
    
    for index, operator in enumerate(operators):
    
        print 'Adding Operator {} to graph'.format(operator)
        if index == 0:            
            source_node_id = ''
        
        else:
            source_node_id = operators[index - 1]
        
        mygraph.add_node(operator,
                         operator, 
                         options[operator], source_node_id)
    
    mygraph.view_graph()
    
    mygraph.run()
    
    
    
def ellipsoid_correction_processing(**kwargs):
   
    options = dict()
    
    operators = ['Read',
                 'Ellipsoid-Correction-RD',
                 'Write']
    
    for operator in operators:
            
        print 'Getting default values for Operator {}'.format(operator)
        parameters = get_operator_default_parameters(operator)
        
        options[operator] = parameters

    for key, value in kwargs.items():
        
        print 'Updating Operator {}'.format(key)
        options[key.replace('_', '-')].update(value)
    
    mygraph = GraphProcessor()
    
    for index, operator in enumerate(operators):
    
        print 'Adding Operator {} to graph'.format(operator)
        if index == 0:            
            source_node_id = ''
        
        else:
            source_node_id = operators[index - 1]
        
        mygraph.add_node(operator,
                         operator, 
                         options[operator], source_node_id)
    
    mygraph.view_graph()
    
    mygraph.run()
    
    
def write_output_image(filepath, output_matrix, image_format, data_format, mask=None, output_projection=None, output_geotransform=None, no_data_value=None):
    
    driver = gdal.GetDriverByName(image_format)
    out_rows = np.size(output_matrix, 0)
    out_columns = np.size(output_matrix, 1)
    
    
    if mask is not None and mask is not 0:
        # TODO: check if output folder exists
        output = driver.Create(filepath, out_columns, out_rows, 2, data_format)
        mask_band = output.GetRasterBand(2)
        mask_band.WriteArray(mask)
        if no_data_value is not None:
            output_matrix[mask > 0] = no_data_value
    else:
        output = driver.Create(filepath, out_columns, out_rows, 1, data_format)
    
    if output_projection is not None:
        output.SetProjection(output_projection)
    if output_geotransform is not None:
        output.SetGeoTransform(output_geotransform)
    
    raster_band = output.GetRasterBand(1)
    if no_data_value is not None:
        raster_band.SetNoDataValue(no_data_value)
       
    # change color of detected pixels to red
    ct = gdal.ColorTable()
    ct.SetColorEntry(1, (255,0,0,255))
    raster_band.SetColorTable(ct)
        
    raster_band.WriteArray(output_matrix)
    
    gdal.Warp(filepath, output, format="GTiff", outputBoundsSRS='EPSG:4326', xRes=output_geotransform[1], yRes=-output_geotransform[5], targetAlignedPixels=True)
    
def get_formatted_date(datetime_str):
    date = datetime.datetime.strftime(datetime_str, '%Y-%m-%dT%H:%M:%SZ')
    return date


def write_properties_file(output_name, first_date, last_date, region_of_interest):
    
    title = 'Output %s' % output_name
    
    first_date = datetime.datetime(year=first_date.year, month=first_date.month, day=first_date.day)
    first_date = first_date + datetime.timedelta(days=0, hours=0, minutes=0, seconds=0)
    first_date = get_formatted_date(first_date)
    
    last_date = datetime.datetime(year=last_date.year, month=last_date.month, day=last_date.day)
    last_date = last_date + datetime.timedelta(days=0, hours=23, minutes=59, seconds=59)
    last_date = get_formatted_date(last_date)

    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (region_of_interest))

#### Aux folders

In [ ]:
output_folder = ''
temp_folder = 'temp'

In [ ]:
if len(output_folder) > 0:
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)
        
if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)

#### Operators definition

In [ ]:
read = dict()

s1meta = "manifest.safe"


input_identifiers = [input_identifier]

for s1path in input_identifiers:

    s1prd = "%s/%s/%s.SAFE/%s" % (data_path, s1path, s1path, s1meta)

read['file'] =  s1prd
#read['formatName'] = 'Sen3_SLSTRL1B_500m'

read

In [ ]:
landseamask = get_operator_default_parameters('Land-Sea-Mask')

for p in landseamask:
    if p == 'shorelineExtension':
        LandSeaMask[p] = '10'

landseamask

In [ ]:
calibration = get_operator_default_parameters('Calibration')

#for p in LandSeaMask:
#    if p == 'shorelineExtension':
#        LandSeaMask[p] = '10'

calibration

In [ ]:
adaptivethresholding = get_operator_default_parameters('AdaptiveThresholding')

for p in adaptivethresholding:
    if p == 'pfa':
        adaptivethresholding[p] = '12.5'

adaptivethresholding

In [ ]:
objectdiscrimination = get_operator_default_parameters('Object-Discrimination')

for p in objectdiscrimination:
    if p == 'minTargetSizeInMeter':
        objectdiscrimination[p] = '30'

objectdiscrimination

In [ ]:
write = dict()

output_path = os.path.join(temp_folder, 'temp')

write['file'] = output_path
write

#### Processing

In [ ]:
vessel_detection_processing(Read=read,
                            Land_Sea_Mask=landseamask,
                            Calibration=calibration,
                            AdaptiveThresholding=adaptivethresholding,
                            Object_Discrimination=objectdiscrimination,
                            Write=write)

#### Conversion to WGS84

##### Detected vessels

In [ ]:
read = dict()

read['file'] =  output_path + '.dim'

In [ ]:
ellipsoidcorrectionrd = dict()


ellipsoidcorrectionrd['sourceBands'] = 'Sigma0_HH_oil_spill_bit_msk'
ellipsoidcorrectionrd['sourceBandNames'] = 'Sigma0_HH_oil_spill_bit_msk'

#ellipsoidcorrectionrd['sourceBands'] = 'Sigma0_HH'
#ellipsoidcorrectionrd['sourceBandNames'] = 'Sigma0_HH'

ellipsoidcorrectionrd['demName'] = 'GETASSE30'
ellipsoidcorrectionrd['imgResamplingMethod'] = 'BILINEAR_INTERPOLATION'

proj = '''GEOGCS["WGS84(DD)", DATUM["WGS84", SPHEROID["WGS84", 6378137.0, 298.257223563]], PRIMEM["Greenwich", 0.0], UNIT["degree", 0.017453292519943295], AXIS["Geodetic longitude", EAST], AXIS["Geodetic latitude", NORTH]]'''

ellipsoidcorrectionrd['mapProjection'] = proj       # comment this line if no need to convert to UTM/WGS84, default is WGS84
ellipsoidcorrectionrd['saveSelectedSourceBand'] = 'true'

ellipsoidcorrectionrd['nodataValueAtSea'] = 'false'

ellipsoidcorrectionrd['pixelSpacingInMeter'] = '25.0'

In [ ]:
write = dict()

output_path2 = os.path.join(temp_folder, 'temp2.tif')

write['file'] = output_path2
write['formatName'] = 'GeoTIFF'

In [ ]:
ellipsoid_correction_processing(Read=read,
                                Ellipsoid_Correction_RD=ellipsoidcorrectionrd,
                                Write=write)

In [ ]:
#output_path2 = 'temp2.tif'

dataset = gdal.Open(output_path2)

product_array = dataset.GetRasterBand(1).ReadAsArray()
projection = dataset.GetProjection()
geotransform = dataset.GetGeoTransform()
no_data_value = dataset.GetRasterBand(1).GetNoDataValue()
data_type = dataset.GetRasterBand(1).DataType

In [ ]:
output_path2

In [ ]:
# date in str
date_str = input_identifiers[0].split('_')[4].split('T')[0]

# date in datetime
dt = datetime.datetime.strptime(date_str, "%Y%m%d")

# output file path
output_tiff_path = os.path.join(output_folder, 'oil_sheen_bit_S1_HH_' + date_str + '.tif')

# writes final tiff image
write_output_image(output_tiff_path, product_array, 'GTiff', 1, mask=None, output_projection=projection, output_geotransform=geotransform, no_data_value=0)

# writes properties file
write_properties_file(output_tiff_path, dt, dt, aoi)

##### SAR image

In [ ]:
read = dict()

read['file'] =  output_path + '.dim'


In [ ]:
ellipsoidcorrectionrd = dict()


#ellipsoidcorrectionrd['sourceBands'] = 'Sigma0_HH_oil_spill_bit_msk'
#ellipsoidcorrectionrd['sourceBandNames'] = 'Sigma0_HH_oil_spill_bit_msk'

ellipsoidcorrectionrd['sourceBands'] = 'Sigma0_HH'
ellipsoidcorrectionrd['sourceBandNames'] = 'Sigma0_HH'

ellipsoidcorrectionrd['demName'] = 'GETASSE30'
ellipsoidcorrectionrd['imgResamplingMethod'] = 'BILINEAR_INTERPOLATION'

proj = '''GEOGCS["WGS84(DD)", DATUM["WGS84", SPHEROID["WGS84", 6378137.0, 298.257223563]], PRIMEM["Greenwich", 0.0], UNIT["degree", 0.017453292519943295], AXIS["Geodetic longitude", EAST], AXIS["Geodetic latitude", NORTH]]'''

ellipsoidcorrectionrd['mapProjection'] = proj       # comment this line if no need to convert to UTM/WGS84, default is WGS84
ellipsoidcorrectionrd['saveSelectedSourceBand'] = 'true'

ellipsoidcorrectionrd['nodataValueAtSea'] = 'false'

ellipsoidcorrectionrd['pixelSpacingInMeter'] = '25.0'

In [ ]:
write = dict()


#output_path2 = os.path.join(output_folder, 'Sigma0_HH_' + date_str + '.tif')
output_path2 = os.path.join(temp_folder, 'Sigma0_HH_' + date_str + '.tif')

write['file'] = output_path2
write['formatName'] = 'GeoTIFF'

In [ ]:
ellipsoid_correction_processing(Read=read,
                                Ellipsoid_Correction_RD=ellipsoidcorrectionrd,
                                Write=write)

In [ ]:
# output file path
output_tiff_path2 = os.path.join(output_folder, 'Sigma0_HH_' + date_str + '.tif')

ds = gdal.Open(output_path2)

product_array = ds.GetRasterBand(1).ReadAsArray()

vmin = np.percentile(product_array[product_array > 0], 5)
vmax = np.percentile(product_array[product_array > 0], 95)

scaleParams = [[vmin, vmax, 0, 255]]

ds = gdal.Translate(output_tiff_path2, ds, outputType = gdal.GDT_Byte, scaleParams=scaleParams)
ds = None

In [ ]:
# writes properties file
write_properties_file(output_tiff_path2, dt, dt, aoi)

#### Remove temporary folder

In [ ]:
shutil.rmtree(temp_folder)

### <a name="license">License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.